# Using Deeplay Blocks

Blocks are the most versatile part of Deeplay. They are designed to make it possible to substitute a base block with any other block that accepts the same input tensor shape and returns the same output tensor shape.

In [1]:
import deeplay as dl
import torch

## Selection of Blocks Available in Deeplay

### `LinearBlock`

You can sequentially add layers to a block ...

In [2]:
block = dl.LinearBlock(4, 10)

print(block)

LinearBlock(
  (layer): Layer[Linear](in_features=4, out_features=10, bias=True)
)


In [3]:
block.activated(torch.nn.ReLU, mode="prepend")

print(block)

LinearBlock(
  (activation): Layer[ReLU]()
  (layer): Layer[Linear](in_features=4, out_features=10, bias=True)
)


In [4]:
block.normalized(torch.nn.LayerNorm, mode="insert", after="layer")

print(block)

LinearBlock(
  (activation): Layer[ReLU]()
  (layer): Layer[Linear](in_features=4, out_features=10, bias=True)
  (normalization): Layer[LayerNorm]()
)


In [5]:
block.shortcut(merge=dl.ops.Add(), shortcut=dl.Layer(torch.nn.Linear, 4, 10))

print(block)

LinearBlock(
  (shortcut_start): Layer[Linear](in_features=4, out_features=10)
  (activation): Layer[ReLU]()
  (layer): Layer[Linear](in_features=4, out_features=10, bias=True)
  (normalization): Layer[LayerNorm]()
  (shortcut_end): Add()
)


In [6]:
block.set_dropout(0.2)

print(block)

LinearBlock(
  (shortcut_start): Layer[Linear](in_features=4, out_features=10)
  (activation): Layer[ReLU]()
  (layer): Layer[Linear](in_features=4, out_features=10, bias=True)
  (normalization): Layer[LayerNorm]()
  (shortcut_end): Add()
  (dropout): Layer[Dropout](p=0.2)
)


... you can also create multiple blocks and modify them at once.

In [7]:
block = dl.LinearBlock(4, 10)

print(block)

LinearBlock(
  (layer): Layer[Linear](in_features=4, out_features=10, bias=True)
)


In [8]:
block.multi(2)  ### The second block should have in_features=10.
print(block)

LinearBlock(
  (blocks): Sequential(
    (0): LinearBlock(
      (layer): Layer[Linear](in_features=4, out_features=10, bias=True)
    )
    (1): LinearBlock(
      (layer): Layer[Linear](in_features=10, out_features=10, bias=True)
    )
  )
)


In [9]:
block["blocks"].all \
    .activated(torch.nn.ReLU) \
    .normalized(torch.nn.LayerNorm, mode="insert", after="layer") \
    .shortcut()

print(block)

LinearBlock(
  (blocks): Sequential(
    (0): LinearBlock(
      (shortcut_start): Layer[Linear](in_features=4, out_features=10)
      (layer): Layer[Linear](in_features=4, out_features=10, bias=True)
      (normalization): Layer[LayerNorm]()
      (activation): Layer[ReLU]()
      (shortcut_end): Add()
    )
    (1): LinearBlock(
      (shortcut_start): Layer[Identity]()
      (layer): Layer[Linear](in_features=10, out_features=10, bias=True)
      (normalization): Layer[LayerNorm]()
      (activation): Layer[ReLU]()
      (shortcut_end): Add()
    )
  )
)


In [10]:
block.set_dropout(0.2)

print(block)

LinearBlock(
  (blocks): Sequential(
    (0): LinearBlock(
      (shortcut_start): Layer[Linear](in_features=4, out_features=10)
      (layer): Layer[Linear](in_features=4, out_features=10, bias=True)
      (normalization): Layer[LayerNorm]()
      (activation): Layer[ReLU]()
      (shortcut_end): Add()
    )
    (1): LinearBlock(
      (shortcut_start): Layer[Identity]()
      (layer): Layer[Linear](in_features=10, out_features=10, bias=True)
      (normalization): Layer[LayerNorm]()
      (activation): Layer[ReLU]()
      (shortcut_end): Add()
    )
  )
  (dropout): Layer[Dropout](p=0.2)
)


### `Conv2dBlock`

In [11]:
block = dl.Conv2dBlock(3, 10, kernel_size=1)

print(block)

Conv2dBlock(
  (layer): Layer[Conv2d](in_channels=3, out_channels=10, kernel_size=1, stride=1, padding=0)
)


In [12]:
block.activated(torch.nn.ReLU) \
    .strided(2) \
    .shortcut() \
    .normalized(torch.nn.LayerNorm, mode="append")

print(block)

Conv2dBlock(
  (shortcut_start): Conv2dBlock(
    (layer): Layer[Conv2d](in_channels=3, out_channels=10, kernel_size=1, stride=2, padding=0)
  )
  (layer): Layer[Conv2d](in_channels=3, out_channels=10, kernel_size=1, stride=2, padding=0)
  (activation): Layer[ReLU]()
  (shortcut_end): Add()
  (normalization): Layer[LayerNorm](normalized_shape=10)
)


In [13]:
block = dl.Conv2dBlock(3, 10, kernel_size=1).upsampled()

print(block)

Conv2dBlock(
  (layer): Layer[Conv2d](in_channels=3, out_channels=10, kernel_size=1, stride=1, padding=0)
  (upsample): Layer[ConvTranspose2d](kernel_size=2, stride=2, padding=0, in_channels=10, out_channels=10)
)


In [14]:
block = dl.Conv2dBlock(3, 10, kernel_size=1).pooled()

print(block)

Conv2dBlock(
  (pool): Layer[MaxPool2d](kernel_size=2, stride=2)
  (layer): Layer[Conv2d](in_channels=3, out_channels=10, kernel_size=1, stride=1, padding=0)
)


### `Sequence1dBlock`

In [15]:
block = dl.Sequence1dBlock(4, 10).LSTM()

print(block)

Sequence1dBlock(
  (layer): Layer[LSTM](input_size=4, hidden_size=10, batch_first=False)
)


In [16]:
block = dl.Sequence1dBlock(4, 10).GRU()

print(block)

Sequence1dBlock(
  (layer): Layer[GRU](input_size=4, hidden_size=10, batch_first=False)
)


In [17]:
block = dl.Sequence1dBlock(4, 10).RNN()

print(block)

Sequence1dBlock(
  (layer): Layer[RNN](input_size=4, hidden_size=10, batch_first=False)
)


## Using the Main Lower-level Block Methods

### Adjusting Layer Order

Blocks work through named layers executed sequentially based on a list containing the layer names ...

In [18]:
block = dl.LinearBlock(4, 10) \
    .activated(torch.nn.ReLU) \
    .normalized()

print(block)

LinearBlock(
  (layer): Layer[Linear](in_features=4, out_features=10, bias=True)
  (activation): Layer[ReLU]()
  (normalization): Layer[BatchNorm1d](num_features=10)
)


... the layer order can be configured.

In [19]:
block.configure(order=["layer", "normalization", "activation"])

print(block)

LinearBlock(
  (layer): Layer[Linear](in_features=4, out_features=10, bias=True)
  (normalization): Layer[BatchNorm1d](num_features=10)
  (activation): Layer[ReLU]()
)


### Adding Layers

You can add layers to a block using various the `.append()`, `.prepend()`, `.insert()`, and `.set()` methods.

#### Adding Layers with the `.append()` Method

In [20]:
block = dl.Conv2dBlock(3, 10, kernel_size=1)

print(block)

Conv2dBlock(
  (layer): Layer[Conv2d](in_channels=3, out_channels=10, kernel_size=1, stride=1, padding=0)
)


In [21]:
block.append(dl.Layer(torch.nn.ReLU))

print(block)

Conv2dBlock(
  (layer): Layer[Conv2d](in_channels=3, out_channels=10, kernel_size=1, stride=1, padding=0)
  (relu): Layer[ReLU]()
)


In [22]:
block.append(dl.Layer(torch.nn.LayerNorm), name="normalization")

print(block)

Conv2dBlock(
  (layer): Layer[Conv2d](in_channels=3, out_channels=10, kernel_size=1, stride=1, padding=0)
  (relu): Layer[ReLU]()
  (normalization): Layer[LayerNorm]()
)


**Note:** The name of the added layer is automatically set to the class name.

#### Adding Layers with the `.prepend()` method

In [23]:
block = dl.Conv2dBlock(3, 10, kernel_size=1)

print(block)

Conv2dBlock(
  (layer): Layer[Conv2d](in_channels=3, out_channels=10, kernel_size=1, stride=1, padding=0)
)


In [24]:
block.prepend(dl.Layer(torch.nn.MaxPool2d, kernel_size=2), name="pool")

print(block)

Conv2dBlock(
  (pool): Layer[MaxPool2d](kernel_size=2)
  (layer): Layer[Conv2d](in_channels=3, out_channels=10, kernel_size=1, stride=1, padding=0)
)


#### Adding Layers with the `.insert()` method

In [25]:
block = dl.Conv2dBlock(3, 10, kernel_size=1).activated(torch.nn.ReLU)

print(block)

Conv2dBlock(
  (layer): Layer[Conv2d](in_channels=3, out_channels=10, kernel_size=1, stride=1, padding=0)
  (activation): Layer[ReLU]()
)


In [26]:
block.insert(dl.Layer(torch.nn.LayerNorm), after="layer", name="normalization")
print(block)

Conv2dBlock(
  (layer): Layer[Conv2d](in_channels=3, out_channels=10, kernel_size=1, stride=1, padding=0)
  (normalization): Layer[LayerNorm]()
  (activation): Layer[ReLU]()
)


#### Adding Layers with the `.set()` method

Unlike previous methods, the `.set()` method replaces the layer with the given name ...

In [27]:
block_with_activation = dl.Conv2dBlock(3, 10, kernel_size=1) \
    .activated(torch.nn.ReLU)

print(block_with_activation)

Conv2dBlock(
  (layer): Layer[Conv2d](in_channels=3, out_channels=10, kernel_size=1, stride=1, padding=0)
  (activation): Layer[ReLU]()
)


In [28]:
block_with_activation.set("activation", torch.nn.ReLU)

print(block_with_activation)

Conv2dBlock(
  (layer): Layer[Conv2d](in_channels=3, out_channels=10, kernel_size=1, stride=1, padding=0)
  (activation): Layer[ReLU]()
)


... if the layer wih the given name already exists.

In [29]:
block_without_activation = dl.Conv2dBlock(3, 10, kernel_size=1)

print(block_without_activation)

Conv2dBlock(
  (layer): Layer[Conv2d](in_channels=3, out_channels=10, kernel_size=1, stride=1, padding=0)
)


In [30]:
block_without_activation.set("activation", torch.nn.ReLU)

print(block_without_activation)

Conv2dBlock(
  (layer): Layer[Conv2d](in_channels=3, out_channels=10, kernel_size=1, stride=1, padding=0)
  (activation): Layer[ReLU]()
)


### Removing Layers with the `.remove()` Method

Layers can be removed using the `.remove()` method, which removes a layer based on its name.

In [31]:
block = dl.Conv2dBlock(3, 10, kernel_size=1).activated(torch.nn.ReLU)

print(block)

Conv2dBlock(
  (layer): Layer[Conv2d](in_channels=3, out_channels=10, kernel_size=1, stride=1, padding=0)
  (activation): Layer[ReLU]()
)


In [32]:
block.remove("activation", allow_missing=True)

print(block)

Conv2dBlock(
  (layer): Layer[Conv2d](in_channels=3, out_channels=10, kernel_size=1, stride=1, padding=0)
)


## Using Operations

Operations are special layers that are implemented in Deeplay directly. They are not blocks, but can be used as layers in blocks. They are used to perform operations that are not implemented in PyTorch, such as `Flatten` or `Reshape`.

### Merge Operations

In [33]:
x = torch.randn(2, 3, 4, 5)
y = torch.randn(2, 3, 4, 5)

merge_add = dl.ops.Add().build()

print(merge_add(x, y).shape)

torch.Size([2, 3, 4, 5])


In [34]:
x = torch.randn(2, 3, 4, 5)
y = torch.randn(2, 3, 4, 5)

merge_cat = dl.ops.Cat(dim=1).build()

print(merge_cat(x, y).shape)

torch.Size([2, 6, 4, 5])


In [35]:
x = torch.randn(2, 3, 4, 5)
y = torch.randn(2, 3, 4, 5)

merge_lambda = dl.ops.Lambda(lambda x: x[0] + x[1]).build()

print(merge_lambda(x, y).shape)

torch.Size([2, 3, 4, 5])


In [36]:
x = torch.randn(2, 3, 4, 5)
y = torch.randn(2, 3, 4, 5)

block = dl.Conv2dBlock(3, 10, kernel_size=1).activated(torch.nn.ReLU)
block.shortcut(merge=merge_cat)

print(block)

Conv2dBlock(
  (shortcut_start): Conv2dBlock(
    (layer): Layer[Conv2d](in_channels=3, out_channels=10, kernel_size=1, stride=1, padding=0)
  )
  (layer): Layer[Conv2d](in_channels=3, out_channels=10, kernel_size=1, stride=1, padding=0)
  (activation): Layer[ReLU]()
  (shortcut_end): Cat()
)


### Reshaping Operations

In [37]:
x = torch.randn(2, 3, 4, 5)

reshape = dl.ops.Reshape(-1, 10)

print(reshape(x).shape)

torch.Size([12, 10])


In [38]:
x = torch.randn(2, 3, 4, 5)

reshape_func = dl.ops.Reshape(lambda shape: (shape[0], -1))

print(reshape_func(x).shape)

torch.Size([2, 60])


In [39]:
x = torch.randn(2, 1, 3, 1)

squeeze = dl.ops.Squeeze(dim=1)

print(squeeze(x).shape)

torch.Size([2, 3, 1])


In [40]:
x = torch.randn(2, 1, 3, 1)

squeeze_all = dl.ops.Squeeze()

print(squeeze_all(x).shape)

torch.Size([2, 3])


In [41]:
x = torch.randn(2, 3, 4, 5)

unsqueeze = dl.ops.Unsqueeze(dim=1)

print(unsqueeze(x).shape)

torch.Size([2, 1, 3, 4, 5])


In [42]:
x = torch.randn(2, 3, 4, 5)

flatten = dl.ops.Flatten(start_dim=1, end_dim=2)

print(flatten(x).shape)

torch.Size([2, 12, 5])


In [43]:
x = torch.randn(2, 3, 4, 5)

permute = dl.ops.Permute(0, 2, 1, 3)

print(permute(x).shape)

torch.Size([2, 4, 3, 5])
